## DATA PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
df = pd.read_excel("category.xlsx",sheet_name= [1,4])
loc_to_cat = df[1]
allUserData = df[4]
#userData = [df[3], df[4], df[5], df[6], df[7]]

#for i in userData[1:]:
#    allUserData = allUserData.append(i)
allUserData.head()

,user_id,date,weekday,start_time,duration,category,category_id,location_name,location_category,my_place,location_id,place_id
0,2,2020-06-29,2,16:00:00,03:00:00,meeting,13,路易莎,cafe,NaN,4.0,NaN
1,2,2020-06-29,2,19:00:00,01:00:00,food,1,全哥鵝肉小炒,restaurant,NaN,5.0,NaN
2,2,2020-06-29,2,20:00:00,00:30:00,home,2,NaN,NaN,home,NaN,1.0
3,2,2020-06-29,2,20:30:00,01:00:00,shopping,12,Miramar,Shopping mall,NaN,6.0,NaN
4,2,2020-06-29,2,21:30:00,02:00:00,study,5,NaN,NaN,home,NaN,1.0


In [3]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','location_name','location_category','my_place','location_id','place_id'], 1)

In [4]:
def to_mins(ts):
    a = ts.hour*60 + ts.minute
    return a

allUserData['start_time'] = allUserData['start_time'].apply(to_mins)
allUserData['duration'] = allUserData['duration'].apply(to_mins)

allUserData.tail()

,user_id,weekday,start_time,duration,category,category_id,location
300,2,7,120,420,sleep,3,1
301,2,7,540,20,home,2,1
302,2,7,560,40,commute,17,99
303,2,7,600,180,hangout,11,4
304,2,7,780,60,food,1,5


In [5]:
cols = allUserData.columns
for col in cols:
    print(col)

user_id
weekday
start_time
duration
category
category_id
location


## Random Forest Model

In [6]:
train_df = allUserData[allUserData['weekday'] < 5]
test_df = allUserData[allUserData['weekday'] >= 5]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 166
Number of testing data is : 139


In [7]:
# without diff
X_train = train_df.iloc[:, np.r_[0:3, 6]].values
y_train = train_df.iloc[:, 5].values

X_test = test_df.iloc[:, np.r_[0:3, 6]].values
y_test = test_df.iloc[:, 5].values

In [8]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [9]:
clf.score(X_test, y_test)

0.7841726618705036

In [10]:
imp = clf.feature_importances_
imp

array([0.        , 0.04307985, 0.29493397, 0.66198618])

In [11]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.56      0.78      0.65        18
           2       1.00      0.92      0.96        24
           3       0.86      1.00      0.92        12
           4       0.50      0.75      0.60         4
           5       0.00      0.00      0.00         1
           6       0.69      0.90      0.78        10
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
           9       0.25      1.00      0.40         2
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00         4
          13       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         3
          17       0.94      0.98      0.96        48
          18       0.00      0.00      0.00         3

   micro avg       0.78      0.78      0.78       139
   macro avg       0.32      0.42      0.35       139
weighted avg       0.71   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
print(confusion_matrix(y_test,predictions))

[[14  0  0  2  0  0  0  0  1  0  0  0  0  1  0]
 [ 0 22  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0 47  0]
 [ 1  0  0  0  0  0  0  0  2  0  0  0  0  0  0]]


In [13]:
predictions

array([ 3,  3,  2, 17,  6,  1,  6, 17,  6, 17,  2,  3,  2, 17,  6,  1,  6,
       17,  1,  4, 17, 17,  2,  3,  2, 17,  1,  1, 17,  4,  1,  9,  9, 17,
        2,  3, 17,  6,  1, 17,  1, 17,  6, 17,  2, 17, 17,  9,  9, 17,  2,
        3,  2, 17,  1, 17,  1, 17,  6, 17, 17,  4, 17,  3,  3,  2, 17,  1,
       17,  2, 17,  1, 17,  9,  2,  3,  2, 17,  6, 17,  6, 17,  1, 17,  2,
       17,  9, 17,  2,  3,  2, 17,  1, 17,  1, 17,  1, 17,  1, 17,  4,  4,
       17,  9,  3,  1, 17,  1,  1,  1, 17,  8, 17,  9,  3,  2, 17,  6, 17,
        2, 17,  6,  1, 17,  2,  3,  2, 17,  6, 17,  1,  4, 17,  2,  3,  2,
       17,  1,  1])

## User Data

In [375]:
location = pd.read_csv("location.csv")
location.head()

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
0,1,121.576000,24.986000,7/22\n\n,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2,c3,c4,c5
1,2,121.443000,24.433000,7/23\n,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2,c3,c4,c5
2,3,121.123000,24.765000,7/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1,c2,c3,c4,c5
3,4,121.516444,25.044673,2020/07/30,15:30,5,0.0,-1.0,Notch 咖啡工場 站前店,50嵐 信陽店,洋庭坊義大利麵,大根堂關東煮/台北車站美食/台北車站小吃/台北車站水煮餐/北車美食/北吃小吃/北車水煮餐,全家便利商店 南陽店,cafe,point_of_interest,restaurant,restaurant,convenience_store
4,5,121.516415,25.044669,2020/07/30,16:59,5,0.0,-1.0,Notch 咖啡工場 站前店,50嵐 信陽店,洋庭坊義大利麵,屈臣氏南陽店,學成k書中心,cafe,point_of_interest,restaurant,drugstore,real_estate_agency


In [376]:
stopLoc = location.drop(['category2','category3','category4','category5'], 1)

def string_to_mins(ts):
    a = int(ts.split(':')[0])
    b = int(ts.split(':')[1])
    c = a*60 + b
    return c

stopLoc['start_time'] = pd.to_datetime(stopLoc['start_time'], format='%H:%M', errors='ignore').apply(string_to_mins)
stopLoc['longitude'] = round(stopLoc['longitude'],3)
stopLoc['latitude'] = round(stopLoc['latitude'],3)
stopLoc.head()

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1
0,1,121.576,24.986,7/22\n\n,660,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1
1,2,121.443,24.433,7/23\n,720,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1
2,3,121.123,24.765,7/24,780,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1
3,4,121.516,25.045,2020/07/30,930,5,0.0,-1.0,Notch 咖啡工場 站前店,50嵐 信陽店,洋庭坊義大利麵,大根堂關東煮/台北車站美食/台北車站小吃/台北車站水煮餐/北車美食/北吃小吃/北車水煮餐,全家便利商店 南陽店,cafe
4,5,121.516,25.045,2020/07/30,1019,5,0.0,-1.0,Notch 咖啡工場 站前店,50嵐 信陽店,洋庭坊義大利麵,屈臣氏南陽店,學成k書中心,cafe


In [377]:
#delete trivial data
dropList = []

for row in range(1,len(stopLoc)-1):   
    
    #compared with the last record, over 2 place name duplicated will be deleted
    res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
    if len(res) > 1: 
        dropList.append(row)
    
    stopLoc.iloc[row,6] = stopLoc.iloc[row+1,4] - stopLoc.iloc[row,4] 
    if stopLoc.iloc[row,6] < 10:
        dropList.append(row)

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.loc[stopLoc['speed'] < 0]
stopLoc = stopLoc.reset_index(drop = True)

In [378]:
#calculate duration
dropList = []
for row in range(0,len(stopLoc)-1): 

    if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
        stopLoc.iloc[row,6] = stopLoc.iloc[row+1,4] - stopLoc.iloc[row,4]   
    else:
        stopLoc.iloc[row,6] = stopLoc.iloc[row+1,4] + 1440 - stopLoc.iloc[row,4]

    if stopLoc.iloc[row,6] < 10:
        dropList.append(row)

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.reset_index(drop = True)
stopLoc = stopLoc.drop(['start_date','name2','name3','name4','name5','speed'], 1)
stopLoc

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1
0,4,121.516,25.045,930,5,89.0,Notch 咖啡工場 站前店,cafe
1,5,121.516,25.045,1019,5,0.0,Notch 咖啡工場 站前店,cafe


## Myplace

In [379]:
savedplace = pd.read_csv("savedPlace.csv")
savedplace['place_longitude'] = round(saveplace['place_longitude'],3)
savedplace['place_latitude'] = round(saveplace['place_latitude'],3)
#savedplace = savedplace.loc[savedplace['my_place'] == 1]
savedplace = savedplace.drop(['place_name','my_place','place_id'],1)
savedplace = savedplace.rename(columns={"place_longitude": "longitude", "place_latitude": "latitude"})
savedplace = savedplace.drop_duplicates(subset=['longitude', 'latitude'], keep='last')
savedplace.head()

,place_category,longitude,latitude
0,university,121.576,24.986
2,NaN,121.516,25.045


In [380]:
stopLoc = pd.merge(stopLoc, savedplace, how='left', on=['longitude','latitude'])
stopLoc

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,place_category
0,4,121.516,25.045,930,5,89.0,Notch 咖啡工場 站前店,cafe,NaN
1,5,121.516,25.045,1019,5,0.0,Notch 咖啡工場 站前店,cafe,NaN


In [381]:
stopLoc = pd.merge(stopLoc, loc_to_cat, how='left', on='category1' )
stopLoc 
#For now, LACKING DATA: user_id
#把start_date加回來
#name1,category1拿掉 place_cat對表換成數字

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,place_category,location
0,4,121.516,25.045,930,5,89.0,Notch 咖啡工場 站前店,cafe,NaN,4.0
1,5,121.516,25.045,1019,5,0.0,Notch 咖啡工場 站前店,cafe,NaN,4.0
